# FullControl design template

*<<< check out demo models [here](https://github.com/FullControlXYZ/fullcontrol/tree/master/models/README.md) >>>*
  
run all cells in this notebook, or press shift+enter to run each cell sequentially 

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*

In [7]:
import fullcontrol as fc
import math

In [8]:
# printer/gcode parameters

design_name = '2D DoE specimen'
nozzle_temp = 0
bed_temp = 0
print_speed = 300   #in mm/min
fan_percent = 0
printer_name='generic' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [9]:
# design parameters

EW = 0.5 # extrusion width
EH = 0.5 # extrusion height (and layer height)
initial_z = EH*1 # initial nozzle position is set to 0.6x the extrusion height to get a bit of 'squish' for good bed adhesion
sq=20 # Length of the square
layers = 2

In [10]:
# generate the design (make sure you've run the above cells before running this cell)

steps = []
for layer in range(layers):
  if layer%2==0:
    steps.append(fc.Point(x=200, y=200, z=initial_z+layer*EH))
    steps.append(fc.Point(x=200, y=200+sq, z=initial_z+layer*EH))
    steps.append(fc.Point(x=200+sq, y=200+sq, z=initial_z+layer*EH))
    steps.append(fc.Point(x=200+sq, y=200, z=initial_z+layer*EH))
    steps.append(fc.Point(x=200+EW, y=200, z=initial_z+layer*EH))
    start_point = fc.Point(x=200+EW, y=200+EW, z=initial_z+layer*EH)
    direction = fc.Vector(x=1,y=0)
    amplitude = sq-(2*EW) # length of the lines 
    line_spacing = 2  # Distance between the lines
    periods = sq//4 # number of times the pattern is repeated
    extra_half_period = False
    extra_end_line = False
    infill = fc.squarewaveXY(start_point, direction, amplitude, line_spacing, periods, extra_half_period, extra_end_line)
    steps.extend(infill)
  else:
    steps.append(fc.Point(x=200+sq, y=200, z=initial_z+layer*EH))
    steps.append(fc.Point(x=200, y=200, z=initial_z+layer*EH))
    steps.append(fc.Point(x=200, y=200+sq, z=initial_z+layer*EH))
    steps.append(fc.Point(x=200+sq, y=200+sq, z=initial_z+layer*EH))
    steps.append(fc.Point(x=200+sq, y=200+EW, z=initial_z+layer*EH))
    start_point = fc.Point(x=200+sq-EW, y=200+EW, z=initial_z+layer*EH)
    direction = fc.Vector(x=0,y=1)
    amplitude = sq-(2*EW) # length of the lines 
    line_spacing = 2 # Distance between the lines
    periods = sq//4 # number of times the pattern is repeated
    extra_half_period = False
    extra_end_line = False
    infill = fc.squarewaveXY(start_point, direction, amplitude, line_spacing, periods, extra_half_period, extra_end_line)
    steps.extend(infill)


In [11]:
# preview the design

fc.transform(steps, 'plot', fc.PlotControls(style='line', zoom=0.7))
# hover the cursor over the lines in the plot to check xyz positions of the points in the design

# uncomment the next line to create a plot with real heights/widths for extruded lines to preview the real 3D printed geometry
fc.transform(steps, 'plot', fc.PlotControls(style='tube', zoom=0.7, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

# uncomment the next line to create a neat preview (click the top-left button in the plot for a .png file) - post and tag @FullControlXYZ :)
fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.5, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))


    - use fc.transform(..., controls=fc.PlotControls(style='tube') to disable this message or style='line' for a simpler line preview



In [12]:
#End Gcode
steps.append(fc.Extruder(on=False))
steps.append(fc.Point(x=200+sq+5,y=200+sq+5,z=4+initial_z+layer*EH))
steps.append(fc.Point(x=0,y=200+sq+5,z=4+initial_z+layer*EH))
steps.append(fc.Point(x=0,y=0,z=0))

# generate and save gcode

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,
    save_as=design_name,
    initialization_data={
        'primer': 'front_lines_then_y',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_percent': fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
gcode = fc.transform(steps, 'gcode', gcode_controls)

#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109. 
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol